In [19]:
#imports for langchain
# from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain_community.document_loaders import DirectoryLoader, TextLoader, JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import glob
import os

In [20]:
text_loader_kwargs = {'autodetect_encoding': True}
folders = [f for f in glob.glob("knowledge-base/**/*", recursive=True) if os.path.isdir(f)]

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    
    # Load .md files
    md_loader = DirectoryLoader(
        folder,
        glob="**/*.md",
        loader_cls=TextLoader,
        loader_kwargs=text_loader_kwargs,
        recursive=True
    )
    md_docs = md_loader.load()
    for doc in md_docs:
        doc.metadata["doc_type"] = doc_type
    documents.extend(md_docs)
    
    # Load .json files, cần truyền jq_schema
    json_loader = DirectoryLoader(
        folder,
        glob="**/*.json",
        loader_cls=JSONLoader,
        loader_kwargs={"jq_schema": ".questions[] | .question + \"\\n\" + .answer"},
        recursive=True
    )
    json_docs = json_loader.load()
    for doc in json_docs:
        doc.metadata["doc_type"] = doc_type
    documents.extend(json_docs)

print("Total documents loaded:", len(documents))


Total documents loaded: 45


In [21]:
documents[0]

Document(metadata={'source': 'knowledge-base/giao_thong/traffic_issue_01.md', 'doc_type': 'giao_thong'}, page_content='# Vấn đề giao thông 1\n\n## Tổng quan\nTrong thời gian gần đây, các vấn đề về giao thông đô thị ngày càng trở nên phức tạp. Nguyên nhân xuất phát từ sự gia tăng số lượng phương tiện, sự xuất hiện của các loại xe mới, và những quy định mới về việc cấm một số loại xe điện tại khu vực trung tâm.\n\n## Các loại xe mới xuất hiện\n- Xe điện cá nhân (e-bike, e-scooter)\n- Xe tự hành (autonomous vehicle)\n- Xe hybrid (lai xăng – điện)\n\n## Quy định về cấm xe điện\nTheo quy định mới, một số khu vực trung tâm sẽ cấm:\n- Xe điện hai bánh không đăng ký biển số\n- Xe điện công suất dưới 250W lưu thông trên đường quốc lộ\n\n## Người liên quan\n- Cơ quan quản lý giao thông\n- Cảnh sát giao thông\n- Người điều khiển phương tiện\n- Doanh nghiệp cung cấp dịch vụ xe điện\n\n## Mức phạt vi phạm (tham khảo)\n| Hành vi | Mức phạt |\n|---------|---------|\n| Điều khiển xe điện không biển số

In [22]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 512, # Slightly smaller chunks for better retrieval
    chunk_overlap=80, # REduced overlap for performance
    separators =["\n\n", "\n", ". ", " ",  ""]
)
chunks = text_splitter.split_documents(documents)
print(f"Create {len(chunks)} chunks")

Create 65 chunks


In [23]:
chunks[7]

Document(metadata={'source': 'knowledge-base/giao_thong/traffic_issue_03.md', 'doc_type': 'giao_thong'}, page_content='## Quy định về cấm xe điện\nTheo quy định mới, một số khu vực trung tâm sẽ cấm:\n- Xe điện hai bánh không đăng ký biển số\n- Xe điện công suất dưới 250W lưu thông trên đường quốc lộ\n\n## Người liên quan\n- Cơ quan quản lý giao thông\n- Cảnh sát giao thông\n- Người điều khiển phương tiện\n- Doanh nghiệp cung cấp dịch vụ xe điện')

In [24]:
doc_type = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Các loại tài liệu đã tìm thấy: {', '.join(doc_type)}")

Các loại tài liệu đã tìm thấy: giao_thong, traffic_law_qa_img_files, test


In [25]:
for chunk in chunks:
    if 'xe' in chunk.page_content:
        print(chunk)
        print("___________")

page_content='# Vấn đề giao thông 1

## Tổng quan
Trong thời gian gần đây, các vấn đề về giao thông đô thị ngày càng trở nên phức tạp. Nguyên nhân xuất phát từ sự gia tăng số lượng phương tiện, sự xuất hiện của các loại xe mới, và những quy định mới về việc cấm một số loại xe điện tại khu vực trung tâm.

## Các loại xe mới xuất hiện
- Xe điện cá nhân (e-bike, e-scooter)
- Xe tự hành (autonomous vehicle)
- Xe hybrid (lai xăng – điện)' metadata={'source': 'knowledge-base/giao_thong/traffic_issue_01.md', 'doc_type': 'giao_thong'}
___________
page_content='## Quy định về cấm xe điện
Theo quy định mới, một số khu vực trung tâm sẽ cấm:
- Xe điện hai bánh không đăng ký biển số
- Xe điện công suất dưới 250W lưu thông trên đường quốc lộ

## Người liên quan
- Cơ quan quản lý giao thông
- Cảnh sát giao thông
- Người điều khiển phương tiện
- Doanh nghiệp cung cấp dịch vụ xe điện' metadata={'source': 'knowledge-base/giao_thong/traffic_issue_01.md', 'doc_type': 'giao_thong'}
___________
page_content

In [26]:
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go

In [27]:
# đưa các đoạn văn bản (chunks) vào Vector Store, liên kết mỗi đoạn với một vector embedding

# embeddings = OpenAIEmbeddings()

# Nếu bạn muốn sử dụng embeddings miễn phí từ Hugging Face ( thay vì Open AI),
# hãy thay dòng embeddings = OpenAIEmbeddings()
# bằng:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


# import os
# os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

# from sentence_transformers import SentenceTransformer
# from langchain_community.embeddings import HuggingFaceEmbeddings

# model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
# embeddings = HuggingFaceEmbeddings(model=model)

# print(embeddings.embed_query("Hello world"))



In [28]:
MODEL = "gpt-oss:20b"
#đặt tên cho vecterdatabase
db_name = "vector_db"
# Kiểm tra nếu database đã tồn tại, thì xóa collection để khởi động lại từ đầu
if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

In [29]:
#tao vector database bang chroma
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=db_name
)
#kiem tra so luong document da duoc luu vao vector store
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 65 documents


In [30]:
#lay ra bo suu tap vector tu vectorstore
collection = vectorstore._collection

#lay 1 embedding tu database
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]

#kiem tra so chieu ( so phan tu cua vector)
dimensions = len(sample_embedding)
print(f"the Vector have {dimensions:,} dimensions")

the Vector have 384 dimensions


In [31]:
sample_embedding

array([-1.03078753e-01,  6.12159222e-02,  1.92221906e-02, -9.71536189e-02,
        1.14988619e-02, -8.57932679e-03,  1.02062777e-01,  9.51457843e-02,
        4.18515392e-02, -3.04474309e-02,  1.41405925e-01, -7.58924633e-02,
        2.39442848e-02, -8.66137166e-03,  2.81839482e-02,  1.73156876e-02,
       -4.15106267e-02,  3.65778208e-02, -8.51145834e-02, -2.94041075e-02,
        2.26024222e-02,  1.93668418e-02,  2.33646668e-02,  3.31485420e-02,
       -6.42463714e-02,  2.01247279e-02,  4.81464807e-03,  6.02077134e-02,
        2.35878713e-02, -8.97502527e-02,  3.71202603e-02,  1.64740771e-01,
       -1.62387919e-02, -5.84476814e-02,  3.81844416e-02,  5.86917857e-03,
       -3.51343974e-02, -9.74030048e-02,  2.80280393e-02, -9.58117843e-03,
       -1.87254660e-02, -7.64859468e-02,  2.38899197e-02, -1.51500359e-01,
        1.00941606e-01,  4.95602451e-02, -2.47762259e-02,  1.65131278e-02,
        7.91175570e-03, -5.80023006e-02, -2.40201037e-02,  4.76933457e-02,
        1.32135050e-02,  

In [33]:
# Lấy toàn bộ vector từ database từ collection
result = collection.get(include=["embeddings", "documents", "metadatas"])

# Đưa embedding vào numpy array
vectors = np.array(result["embeddings"])

# Lưu lại văn bản
documents = result["documents"]

# Trích loại tài liệu từ metadata
doc_types = [metadata.get('doc_type', 'unknown') for metadata in result["metadatas"]]

# Mapping màu cho từng loại tài liệu
color_map = {
    'giao_thong': 'blue',
    'traffic_law_qa_img_files': 'green'
}

# Nếu loại tài liệu không có trong map thì gán 'gray'
colors = [color_map.get(t, 'gray') for t in doc_types]
